# NLP - Spell Checker

- **Created by Andrés Segura Tinoco**
- **Created on June 14, 2019**

A **Spell checker** (or spell corrector) is an application, program or a feature of a program that checks for misspellings in a text and offers possible solutions (candidate words) <a href="#link_one">[1]</a>.

A basic spell checker carries out the following processes:
- It scans the text and extracts the words contained in it.
- It then compares each word with a known list of correctly spelled words (i.e. a dictionary).
- An additional step is a language-dependent algorithm for handling morphology.
- The program's user interface will allow users to approve or reject replacements and modify the program's operation.

## 1. Spell Checker from Scratch

In [1]:
# Load Python libraries
import io
import re

### Create dictionary/vocabulary from a book

In [2]:
# Util function to read a plain text file
def read_text_file(file_path):
    text = ""
    with io.open(file_path, 'r', encoding = 'ISO-8859-1') as f:
        text = f.read()
    return text;

In [3]:
# Get text sample
file_path = "../data/en/Moby Dick or the Whale - Herman Melville.txt"
plain_text = read_text_file(file_path)
len(plain_text)

1235134

In [4]:
# Return an array with the clean words of the document: vocabulary
def get_vocabulary(plain_text):
    vocabulary = []
    doc_words = []
    
    # Cleaing the text
    clean_text = plain_text.lower()
    clean_text = clean_text.replace('\n', '.')
    clean_text = re.sub('[^a-zA-Z.]', ' ', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = re.sub(r'\.+', ".", clean_text)

    # Tokenize sentences in words
    for sentence in clean_text.split('.'):
        if sentence.strip() != '':
            for word in sentence.split():
                if word.strip() != '':
                    doc_words.append(word.strip())
    
    # Get unique words from document
    vocabulary = list(set(doc_words))
    
    return vocabulary;

In [5]:
# Create vocabulary
vocabulary = get_vocabulary(plain_text)
len(vocabulary)

16967

In [6]:
# Show 100 first words of the vocabulary
print(vocabulary[0:100])

['lists', 'enemy', 'unbegotten', 'strict', 'begging', 'hindmost', 'derive', 'judith', 'taken', 'uncracked', 'relinquish', 'slapped', 'resist', 'ceremonies', 'childish', 'sixteen', 'explain', 'tips', 'questionable', 'cabalistics', 'inquiry', 'yawed', 'community', 'changing', 'ensuing', 'actions', 'blubber', 'riveted', 'freedom', 'symbolized', 'placards', 'plates', 'lawful', 'horse', 'feathering', 'cable', 'transpired', 'froissart', 'abednego', 'arrive', 'lockers', 'watergate', 'snatching', 'composing', 'predicament', 'miguel', 'equator', 'voyagings', 'banterings', 'thereabouts', 'bellied', 'heavers', 'faded', 'canopy', 'revolvingly', 'reason', 'subdivisible', 'animosity', 'disclosed', 'trace', 'abroad', 'swallowed', 'chinks', 'surrender', 'bag', 'wondering', 'twine', 'echoed', 'noah', 'guilt', 'leaking', 'such', 'fish', 'anglo', 'dearest', 'nimbly', 'scornest', 'quaffed', 'twopence', 'errors', 'colourless', 'lay', 'drawing', 'stratagem', 'owners', 'continue', 'beggar', 'retaken', 'daugh

### Create Spell Checker model

Based on Peter Norvig’s 21-line spelling corrector using probability theory <a href="#link_two" >[2]</a>.

In [7]:
# Define Spell Checker class
class SpellChecker:
    
    def __init__(self, words):
        "Constructor."
        self.w_rank = {}
        self.letters = 'abcdefghijklmnopqrstuvwxyz'
        
        for i, word in enumerate(words):
            self.w_rank[word] = i
    
    def P(self, word): 
        "Probability of 'word'."
        # use inverse of rank as proxy
        # returns 0 if the word isn't in the dictionary
        return - self.w_rank.get(word, 0)

    def known(self, words): 
        "The subset of 'words' that appear in the dictionary of w_rank."
        return set(w for w in words if w in self.w_rank)

    def edits1(self, word):
        "All edits that are one edit away from 'word'."
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from 'word'."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))
    
    def correction(self, word):
        "Most probable spelling correction for word."
        return max(self.candidates(word), key = self.P)
    
    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

### Test Spell Checker model

In [8]:
# Create model
sp_model = SpellChecker(vocabulary)

In [9]:
# Correction
sp_model.correction('castli')

'castle'

In [10]:
# Get candidates
sp_model.candidates('wlak')

{'walk', 'weak'}

## 2. Spell Checker using PySpellChecker

Pure Python Spell Checking based on Peter Norvig’s blog post on setting up a simple spell checking algorithm <a href="#link_three">[3]</a>.

It uses a Levenshtein Distance algorithm to find permutations within an edit distance of 2 from the original word. It then compares all permutations (insertions, deletions, replacements, and transpositions) to known words in a word frequency list. Those words that are found more often in the frequency list are more likely the correct results. <a href="#link_four">[4]</a>

In [11]:
# Load Python libraries
from spellchecker import SpellChecker

#### English model

In [12]:
# Create English model
spell = SpellChecker(language = 'en')

In [13]:
# Default Levenshtein distance
spell.distance

2

In [14]:
# Find those words that may be misspelled
misspelled = spell.unknown(['She', 'livis', 'in', 'a', 'beaotipul', 'castli'])
misspelled

{'beaotipul', 'castli', 'livis'}

In [15]:
# Get the one 'most likely' answer and options
for word in misspelled:
    solution = {'word': word, 'answer': spell.correction(word), 'options': spell.candidates(word)}
    print(solution)

{'word': 'castli', 'answer': 'castle', 'options': {'castle'}}
{'word': 'livis', 'answer': 'lives', 'options': {'livid', 'levis', 'divis', 'livia', 'lives'}}
{'word': 'beaotipul', 'answer': 'beautiful', 'options': {'beautiful'}}


#### Spanish model

In [16]:
# Create Spanish model
spell = SpellChecker(language = 'es', distance = 1)

In [17]:
# Find those words that may be misspelled
misspelled = spell.unknown(['Esta', 'es', 'una', 'frace', 'con', 'errodes'])
misspelled

{'errodes', 'frace'}

In [18]:
# Get the one 'most likely' answer and options
for word in misspelled:
    solution = {'word': word, 'answer': spell.correction(word), 'options': spell.candidates(word)}
    print(solution)

{'word': 'errodes', 'answer': 'errores', 'options': {'errores'}}
{'word': 'frace', 'answer': 'grace', 'options': {'face', 'race', 'frac', 'frane', 'frase', 'france', 'trace', 'brace', 'frame', 'grace'}}


### Reference

<a name='link_one'   href='https://en.wikipedia.org/wiki/Spell_checker' target='_blank' >[1]</a> Wikipedia Spell Checker.  
<a name='link_two'   href='https://norvig.com/spell-correct.html'       target='_blank' >[2]</a> Peter Norvig Spell Correct.  
<a name='link_three' href='https://pypi.org/project/pyspellchecker/'    target='_blank' >[3]</a> PySpellChecker project.  
<a name='link_four' href='https://ansegura7.github.io/NLP/support/pyspellchecker_manual.pdf' target='_blank' >[4]</a> PySpellChecker PDF manual.  


<hr>
<p><a href="https://ansegura7.github.io/NLP/">« Home</a></p>